In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
url = "https://www.suryaa.com/"

## Extraction of Links

### Setting Up the Driver

In [18]:
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())
except:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")

In [20]:
driver.get(url)

In [21]:
header= driver.find_elements(By.CLASS_NAME, "navbar-nav")
header = header[0].find_elements(By.TAG_NAME, "a")
headerlinks = {i.get_attribute('href') for i in header}
headerlinks -= {"https://www.suryaa.com/index.html","https://www.suryaa.com/index.html#"}
headerlinks = list(headerlinks)

In [34]:
headerlinks

['https://telugu.suryaa.com/telangana-latest.html']

In [33]:
headerlinks = headerlinks[:-1]

### Link Extraction

In [40]:
links = []

In [ ]:
for i in headerlinks:
    driver.get(i)
    while(True):
        divs = driver.find_elements(By.CLASS_NAME, "media-body")
        if(len(divs)==0): break
        for j in divs:
            links.append(j.find_element(By.TAG_NAME, "a").get_attribute('href'))
        next_button = driver.find_elements(By.CLASS_NAME, "pagination")[0].find_elements(By.TAG_NAME, "li")
        # print(len(next_button))
        if(len(next_button)!=3): break
        next_button = next_button[-1].find_elements(By.TAG_NAME, "a")
        if(len(next_button)!=2): break
        next_button[1].click()
    

In [ ]:
driver.close()

In [42]:
df = pd.DataFrame(links, columns=["url"])

In [43]:
df

,url
0,https://telugu.suryaa.com/telangana-news-48989...
1,https://telugu.suryaa.com/telangana-news-48989...
2,https://telugu.suryaa.com/telangana-news-48989...
3,https://telugu.suryaa.com/telangana-news-48989...
4,https://telugu.suryaa.com/telangana-news-48989...
...,...
7995,https://telugu.suryaa.com/telangana-news-48188...
7996,https://telugu.suryaa.com/telangana-news-48188...
7997,https://telugu.suryaa.com/telangana-news-48188...
7998,https://telugu.suryaa.com/telangana-news-48188...


In [67]:
df.head()

,url
0,https://telugu.suryaa.com/telugu-latest-update...
1,https://telugu.suryaa.com/telugu-latest-update...
2,https://telugu.suryaa.com/telugu-latest-update...
3,https://telugu.suryaa.com/telugu-latest-update...
4,https://telugu.suryaa.com/telugu-latest-update...


In [79]:
len(df)

1795

In [44]:
df.to_csv("./Links/surya.csv", index=False, header=False, mode='a')

## Extracting the Text Data from the Saved Links

In [3]:
links = pd.read_csv("./Links/surya.csv")["url"].tolist()

In [4]:
len(links)

57526

### Getting the Driver Ready

In [5]:
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())
except:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")

### Creating the Folder

In [6]:
import os
if not os.path.exists("./Texts/suryaa"):
    os.makedirs("./Texts/suryaa")

### Creating a File for each Article and Storing the Text in the File

In [10]:
for i in range(4402,len(links)):
    try:
        driver.get(links[i])
    except:
        print(f"Could not open the link: {i,links[i]}")
        continue
    try:
        f = os.open(f"./Texts/suryaa/{i}.txt", os.O_RDWR | os.O_CREAT)
        txt = driver.find_element(By.TAG_NAME, "h1").text
        txt += "\n"
        data = driver.find_elements(By.TAG_NAME, "p")[:-1]
        txt += "\n".join([i.text for i in data])
        os.write(f, txt.encode())
        os.close(f)
    except:
        print(f"Could not write the text in the link: {i,links[i]}")
        continue

Could not write the text in the link: (11945, 'https://telugu.suryaa.com/telugu-latest-updates-126402-.html')


In [ ]:
driver.close()